In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.1.0'

In [2]:

BATCH_SIZE=512 #大概需要2G的显存
EPOCHS=20 # 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多

In [3]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

100%|███████████████████████████▉| 9904128/9912422 [00:19<00:00, 931365.82it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz



0it [00:00, ?it/s]
 57%|██████████████████▋              | 16384/28881 [00:00<00:00, 62057.03it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz




0it [00:00, ?it/s]

  0%|                                              | 0/1648877 [00:00<?, ?it/s]

  1%|▎                              | 16384/1648877 [00:00<00:28, 56688.81it/s]

  3%|▉                              | 49152/1648877 [00:01<00:23, 68362.46it/s]

  6%|█▊                             | 98304/1648877 [00:01<00:18, 84642.94it/s]

 10%|██▉                          | 163840/1648877 [00:01<00:14, 105551.81it/s]

 16%|████▌                        | 262144/1648877 [00:01<00:09, 143648.79it/s]

 21%|██████                       | 344064/1648877 [00:01<00:07, 182939.38it/s]

 27%|███████▊                     | 442368/1648877 [00:01<00:04, 241680.11it/s]

 33%|█████████▌                   | 540672/1648877 [00:02<00:03, 311158.91it/s]

 39%|███████████▏                 | 638976/1648877 [00:02<00:02, 390945.76it/s]

 45%|████████████▉                | 737280/1648877 [00:02<00:01, 475776.51it/s]

 51%|██████████████▋              | 835584/1648877 [00:02<00:01, 560027.27it/s]

 57%|█

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz




0it [00:00, ?it/s]

  0%|                                                 | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 14866.67it/s]                                                   

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


9920512it [00:30, 931365.82it/s]                                               

In [4]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)


32768it [00:20, 62057.03it/s]                                                  

In [5]:

class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

In [6]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [8]:

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.251622
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.187996
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.096146

Test set: Average loss: 0.0905, Accuracy: 9733/10000 (97%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.073066
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.074291
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.042246

Test set: Average loss: 0.0505, Accuracy: 9836/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.029856
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.050501
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.050483

Test set: Average loss: 0.0351, Accuracy: 9873/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.021359
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.030007
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.041102

Test set: Average loss: 0.0354, Accuracy: 9871/10000 (99%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.019379
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.041251
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.035415

Test 